In [ ]:
# !pip install biopython

In [1]:
from Bio import Entrez, SeqIO
from tqdm import tqdm
import pandas as pd
import time

In [8]:
Entrez.email = "your_email@example.com"

In [3]:
data = pd.read_csv("../train_data/merge_train_preprocess_refine_columns.csv")

# 개별 검색

In [32]:
data.columns

Index(['SUBCLASS', 'ID', 'BRCA1', 'HMGB3', 'PLXNB2', 'NOTCH2', 'RYR2', 'ALB',
       'BTG1', 'SMC1A',
       ...
       'PEX6', 'B2M', 'MYLK', 'BTG2', 'CACNA1B', 'PABPC1', 'FGFR3', 'CLIP2',
       'CDH1', 'HELZ2'],
      dtype='object', length=150)

In [43]:
# 검색할 단백질 이름을 입력합니다
protein_name = "BRCA1"

In [48]:
search_query = f"{protein_name}[gene] AND Homo sapiens[organism] AND srcdb_refseq[properties]"

In [49]:
# Step 1: 단백질 이름으로 NCBI protein 데이터베이스 검색
try:
    handle = Entrez.esearch(db="protein", term=search_query, retmax=1)
    record = Entrez.read(handle)
    protein_ids = record["IdList"]

    # 단백질 ID가 존재할 경우에만 서열을 가져옵니다
    if protein_ids:
        protein_id = protein_ids[0]
        print(f"Found Protein ID: {protein_id}")
        
        # Step 2: 단백질 서열 가져오기
        handle = Entrez.efetch(db="protein", id=protein_id, rettype="fasta", retmode="text")
        seq_record = SeqIO.read(handle, "fasta")
        
        # 단백질 서열 출력
        print("Protein Name:", protein_name)
        print("Description:", seq_record.description)
        print("Sequence:", seq_record.seq)
        
    else:
        print("No protein found with the given name.")
except Exception as e:
    print(f"An error occurred: {e}")

No protein found with the given name.


In [42]:
protein_ids

['71143137']

In [ ]:
str(seq_record.seq)

In [23]:

# Step 1: 단백질 이름으로 NCBI protein 데이터베이스 검색
try:
    handle = Entrez.esearch(db="protein", term=search_query, retmax=1)
    record = Entrez.read(handle)
    protein_ids = record["IdList"]

    # 단백질 ID가 존재할 경우에만 서열을 가져옵니다
    if protein_ids:
        protein_id = protein_ids[0]
        print(f"Found Protein ID: {protein_id}")
        
        # Step 2: 단백질 서열 가져오기
        handle = Entrez.efetch(db="protein", id=protein_id, rettype="fasta", retmode="text")
        seq_record = SeqIO.read(handle, "fasta")
        
        # 단백질 서열 출력
        print("Protein Name:", protein_name)
        print("Description:", seq_record.description)
        print("Sequence:", seq_record.seq)
    else:
        print("No protein found with the given name.")
except Exception as e:
    print(f"An error occurred: {e}")


No protein found with the given name.


In [19]:
# Step 1: 단백질 이름으로 NCBI protein 데이터베이스 검색
try:
    handle = Entrez.esearch(db="protein", term=protein_name, retmax=1)
    record = Entrez.read(handle)
    protein_ids = record["IdList"]

    # 단백질 ID가 존재할 경우에만 서열을 가져옵니다
    if protein_ids:
        protein_id = protein_ids[0]
        print(f"Found Protein ID: {protein_id}")
        
        # Step 2: 단백질 서열 가져오기
        handle = Entrez.efetch(db="protein", id=protein_id, rettype="fasta", retmode="text")
        seq_record = SeqIO.read(handle, "fasta")
        
        # 단백질 서열 출력
        print("Protein Name:", protein_name)
        print("Description:", seq_record.description)
        print("Sequence:", seq_record.seq)
    else:
        print("No protein found with the given name.")
except Exception as e:
    print(f"An error occurred: {e}")


Found Protein ID: 2813727314
Protein Name: BRCA2
Description: XP_068818217.1 lys-63-specific deubiquitinase BRCC36 [Capricornis sumatraensis]
Sequence: MAVQVVQAVQAVHLESDAFLVCLNHALSTEKEEVMGLCIGELNDDLRNDPKFTYTGTEMRTVAEKVDTVRIVHIHSVIILRRSDKRKDRVEISPEQLSAASTEAERLAELTGRPMRVVGWYHSHPHITVWPSHVDVRTQAMYQMMDQGFVGLIFSCFIEDKNTKTGRVLYTCFQSIQAQKSSESPRGPRDFWSSSQHISIEGQKEEERYERIEIPIHIVPHVTIGKVCLESAVELPKILCQEEQDAYRRIHSLTHLDSVTKIHNGSVFTKNLCSQMSAVSGPLLQWLEDRLEQNQQHVQELQQEKEELLQELSSLE


# refine_col로 추출

In [10]:
import ssl
from tqdm import tqdm
from Bio import Entrez, SeqIO
import time

# SSL 인증서 검증 비활성화
ssl._create_default_https_context = ssl._create_unverified_context

In [50]:
# 단백질 이름 컬럼들을 추출
protein_columns = data.columns[2:]  # ID와 SUBCLASS 열 제외

# 단백질 서열을 저장할 딕셔너리 생성
protein_sequences = {protein: [] for protein in protein_columns}
non_seq = []
# 각 단백질에 대해 NCBI에서 서열 검색 후 딕셔너리에 추가
for protein in tqdm(protein_columns):
    search_query = f"{protein}[gene] AND Homo sapiens[organism] AND srcdb_refseq[properties]"
    try:
        # NCBI protein 데이터베이스에서 검색
        handle = Entrez.esearch(db="protein", term=search_query, retmax=1)
        record = Entrez.read(handle)
        protein_ids = record["IdList"]

        # 단백질 ID가 존재하는 경우 서열 가져오기
        if protein_ids:
            protein_id = protein_ids[0]
            handle = Entrez.efetch(db="protein", id=protein_id, rettype="fasta", retmode="text")
            seq_record = SeqIO.read(handle, "fasta")
            protein_sequences[protein] = seq_record.seq
        else:
            protein_sequences[protein] = None
            print(f"{protein}은 서열 정보가 없습니다.")
            non_seq.append(protein)

        # 너무 많은 요청을 보내지 않도록 잠시 대기
        time.sleep(1)
    except Exception as e:
        protein_sequences[protein] = None
        print(f"Error fetching sequence for {protein}: {e}")



100%|██████████| 148/148 [12:56<00:00,  5.25s/it]


In [51]:
# DataFrame에 단백질 서열 추가
df_sample = pd.DataFrame([protein_sequences])
df_sample.head()

,BRCA1,HMGB3,PLXNB2,NOTCH2,RYR2,ALB,BTG1,SMC1A,AQP7,PIM1,...,PEX6,B2M,MYLK,BTG2,CACNA1B,PABPC1,FGFR3,CLIP2,CDH1,HELZ2
0,"(M, D, L, S, A, L, R, V, E, E, V, Q, N, V, I, ...","(M, A, K, G, D, P, K, K, P, K, G, K, M, S, A, ...","(M, L, L, E, V, S, Y, G, G, S, P, V, P, N, P, ...","(M, P, A, L, R, P, A, L, L, W, A, L, L, A, L, ...","(M, A, D, G, G, E, G, E, D, E, I, Q, F, L, R, ...","(M, K, W, V, T, F, I, S, L, L, F, L, F, S, S, ...","(M, H, P, F, Y, T, R, A, A, T, M, I, G, E, I, ...","(M, L, E, V, S, I, P, T, P, H, R, Y, V, R, G, ...","(M, V, Q, A, S, G, H, R, R, S, T, R, G, S, K, ...","(M, Q, D, L, G, D, K, L, R, M, I, V, M, G, H, ...",...,"(M, A, L, A, V, L, R, V, L, E, P, F, P, T, E, ...","(M, S, R, S, V, A, L, A, V, L, A, L, L, S, L, ...","(M, G, D, V, K, L, V, A, S, S, H, I, S, K, T, ...","(M, S, H, G, K, G, T, D, M, L, P, E, I, A, A, ...","(M, V, R, F, G, D, E, L, G, G, R, Y, G, G, P, ...","(M, N, P, S, A, P, S, Y, P, M, A, S, L, Y, V, ...","(M, G, A, P, A, C, A, L, A, L, C, V, A, V, A, ...","(M, Q, K, P, S, G, L, K, P, P, G, R, G, G, K, ...","(M, A, L, E, V, G, D, Y, K, I, N, L, K, L, M, ...","(M, A, V, W, E, A, E, Q, L, G, G, L, Q, R, G, ..."


In [52]:
# 최종 결과를 CSV 파일로 저장
df_sample.to_csv('../train_data/protein_sequences_output_sample_refine_col.csv', index=False)

# 전체 col로 추출

In [ ]:
train = pd.read_csv("../train_data/train.csv")

In [55]:
# 단백질 이름 컬럼들을 추출
protein_columns = train.columns[2:]  # ID와 SUBCLASS 열 제외

# 단백질 서열을 저장할 딕셔너리 생성
protein_sequences = {protein: [] for protein in protein_columns}
non_seq = []
# 각 단백질에 대해 NCBI에서 서열 검색 후 딕셔너리에 추가
for protein in tqdm(protein_columns):
    try:
        # NCBI protein 데이터베이스에서 검색
        handle = Entrez.esearch(db="protein", term=protein, retmax=1)
        record = Entrez.read(handle)
        protein_ids = record["IdList"]

        # 단백질 ID가 존재하는 경우 서열 가져오기
        if protein_ids:
            protein_id = protein_ids[0]
            handle = Entrez.efetch(db="protein", id=protein_id, rettype="fasta", retmode="text")
            seq_record = SeqIO.read(handle, "fasta")
            protein_sequences[protein] = seq_record.seq
        else:
            protein_sequences[protein] = None
            print(f"{protein}은 서열 정보가 없습니다.")
            non_seq.append(protein)

        # 너무 많은 요청을 보내지 않도록 잠시 대기
        time.sleep(1)
    except Exception as e:
        protein_sequences[protein] = None
        print(f"Error fetching sequence for {protein}: {e}")



NameError: name 'train' is not defined

In [ ]:
# DataFrame에 단백질 서열 추가
df = pd.DataFrame([protein_sequences])
df.head()

In [ ]:

# 최종 결과를 CSV 파일로 저장
df.to_csv('../train_data/protein_sequences_output.csv', index=False)

# Uniprot DB 이용

In [21]:
def extract_sequence(fasta_data):
    lines = fasta_data.strip().split("\n")
    sequence = []
    
    # 헤더 줄을 제외하고 아미노산 서열을 리스트에 추가
    for line in lines:
        if not line.startswith(">"):  # 헤더 줄 무시
            sequence.append(line.strip())
    
    # 모든 아미노산 서열을 하나의 문자열로 결합
    return "".join(sequence)



MPALRPALLWALLALWLCCAAPAHALQCRDGYEPCVNEGMCVTYHNGTGYCKCPEGFLGEYCQHRDPCEKNRCQNGGTCVAQAMLGKATCRCASGFTGEDCQYSTSHPCFVSRPCLNGGTCHMLSRDTYECTCQVGFTGKECQWTDACLSHPCANGSTCTTVANQFSCKCLTGFTGQKCETDVNECDIPGHCQHGGTCLNLPGSYQCQCPQGFTGQYCDSLYVPCAPSPCVNGGTCRQTGDFTFECNCLPGFEGSTCERNIDDCPNHRCQNGGVCVDGVNTYNCRCPPQWTGQFCTEDVDECLLQPNACQNGGTCANRNGGYGCVCVNGWSGDDCSENIDDCAFASCTPGSTCIDRVASFSCMCPEGKAGLLCHLDDACISNPCHKGALCDTNPLNGQYICTCPQGYKGADCTEDVDECAMANSNPCEHAGKCVNTDGAFHCECLKGYAGPRCEMDINECHSDPCQNDATCLDKIGGFTCLCMPGFKGVHCELEINECQSNPCVNNGQCVDKVNRFQCLCPPGFTGPVCQIDIDDCSSTPCLNGAKCIDHPNGYECQCATGFTGVLCEENIDNCDPDPCHHGQCQDGIDSYTCICNPGYMGAICSDQIDECYSSPCLNDGRCIDLVNGYQCNCQPGTSGVNCEINFDDCASNPCIHGICMDGINRYSCVCSPGFTGQRCNIDIDECASNPCRKGATCINGVNGFRCICPEGPHHPSCYSQVNECLSNPCIHGNCTGGLSGYKCLCDAGWVGINCEVDKNECLSNPCQNGGTCDNLVNGYRCTCKKGFKGYNCQVNIDECASNPCLNQGTCFDDISGYTCHCVLPYTGKNCQTVLAPCSPNPCENAAVCKESPNFESYTCLCAPGWQGQRCTIDIDECISKPCMNHGLCHNTQGSYMCECPPGFSGMDCEEDIDDCLANPCQNGGSCMDGVNTFSCLCLPGFTGDKCQTDMNECLSEPCKNGGTCSDYVNSYTCKCQAGFDGVHCENNINECTESSCFNGG

In [ ]:
import requests
protein_columns = data.columns[2:]  # ID와 SUBCLASS 열 제외

# 단백질 서열을 저장할 딕셔너리 생성
protein_sequences = {protein: [] for protein in protein_columns}
non_seq = []
for protein in tqdm(protein_columns):
    prot = f'{protein}'

    # UniProt API URL 및 검색 쿼리 설정
    uniprot_url = "https://rest.uniprot.org/uniprotkb/search"
    params = {
        "query": f"(gene:{prot}) AND (organism_id:9606) AND (reviewed:true)",  # A2M, human (organism_id: 9606), reviewed (Swiss-Prot)
        "format": "fasta",  # FASTA 형식으로 데이터를 가져옴
        "fields": "sequence"  # 서열만 가져오기
    }


    # UniProt에 API 요청
    response = requests.get(uniprot_url, params=params)

    # 요청이 성공했는지 확인
    if response.status_code == 200:
        protein_info = response.text  # FASTA 형식으로 데이터 수신
        
    else:
        print(f"Error: Unable to fetch data (status code {response.status_code})")
    amino_acid_sequence = extract_sequence(protein_info)
    protein_sequences[protein] = amino_acid_sequence

In [24]:
df_sample = pd.DataFrame([protein_sequences])
df_sample.head()

,BRCA1,HMGB3,PLXNB2,NOTCH2,RYR2,ALB,BTG1,SMC1A,AQP7,PIM1,...,PEX6,B2M,MYLK,BTG2,CACNA1B,PABPC1,FGFR3,CLIP2,CDH1,HELZ2
0,MDLSALRVEEVQNVINAMQKILECPICLELIKEPVSTKCDHIFCKF...,MAKGDPKKPKGKMSAYAFFVQTCREEHKKKNPEVPVNFAEFSKKCS...,MALQLWALTLLGLLGAGASLRPRKLDFFRSEKELNHLAVDEASGVV...,MPALRPALLWALLALWLCCAAPAHALQCRDGYEPCVNEGMCVTYHN...,MADGGEGEDEIQFLRTDDEVVLQCTATIHKEQQKLCLAAEGFGNRL...,MAPKTKKGCKVTLPEKPVKLASHTRDTTGVSQMFPSSKARTKSLLG...,MHPFYTRAATMIGEIAAAVSFISKFLRTKGLTSERQLQTFSQSLQE...,MGFLKLIEIENFKSYKGRQIIGPFQRFTAIIGPNGSGKSNLMDAIS...,MVQASGHRRSTRGSKMVSWSVIAKIQEILQRKMVREFLAEFMSTYV...,MLLSKINSLAHLRAAPCNDLHATKLAPGKEKEPLESQYQVGPLLGS...,...,MALAVLRVLEPFPTETPPLAVLLPPGGPWPAAELGLVLALRPAGES...,MSRSVALAVLALLSLSGLEAIQRTPKIQVYSRHPAENGKSNFLNCY...,MGDVKLVASSHISKTSLSVDPSRVDSMPLTEAPAFILPPRNLCIKE...,MSHGKGTDMLPEIAAAVGFLSSLLRTRGCVSEQRLKVFSGALQEAL...,MVRFGDELGGRYGGPGGGERARGGGAGGAGGPGPGGLQPGQRVLYK...,MNPSAPSYPMASLYVGDLHPDVTEAMLYEKFSPAGPILSIRVCRDM...,MGAPACALALCVAVAIVAGASSESLGTEQRVVGRAAEVPGPEPGQQ...,MQKPSGLKPPGRGGKHSSPMGRTSTGSASSSAAVAASSKEGSPLHK...,MGPWSRSLSALLLLLQVSSWLCQEPEPCHPGFDAESYTFTVPRRHL...,MAPPGSTLLPNSPAATRGPSLARLCALVDLCLGCSRCTQRLNESTY...


In [25]:
# 최종 결과를 CSV 파일로 저장
df_sample.to_csv('../train_data/protein_sequences_output_sample_refine_col_uniprot.csv', index=False)